<a href="https://colab.research.google.com/github/juliocnsouzadev/notebooks/blob/develop/Tensorflow_2_0_CNN_Animals_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cow vs Horse Classifier 

## Import Libs

In [ ]:
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from google.colab import drive
import glob

In [ ]:
tf.__version__

In [ ]:
drive.mount('/content/drive')

## Preprocessing Data

### Definitions

In [ ]:
rescale_default = 1./255
shear_range_default=0.2
zoom_range_default=0.2
target_size_default=(64, 64)
batch_size_default=32
class_mode_default='binary'
input_shape_default = [target_size_default[0], target_size_default[1], 3]
main_path = '/content/drive/My Drive/Data Science/datasets/cows-and-horses'
train_folder = main_path + '/train'
test_folder = main_path + '/test'
validaton_folder = main_path + '/validation'
epochs_default = 25

In [ ]:
def preprocess():
  train_datagen = image.ImageDataGenerator(
          rescale=rescale_default,
          shear_range= shear_range_default,
          zoom_range=zoom_range_default,
          horizontal_flip=True)
  train_set = train_datagen.flow_from_directory(
          train_folder,
          target_size=target_size_default,
          batch_size=batch_size_default,
          class_mode=class_mode_default)
  
  test_datagen = image.ImageDataGenerator(rescale=rescale_default)
  test_set = test_datagen.flow_from_directory(
        test_folder,
        target_size=target_size_default,
        batch_size=batch_size_default,
        class_mode=class_mode_default)
  return train_set, test_set

### Get Test And Train Sets

In [ ]:
test_set, train_set = preprocess()

## Init Model and Layers

### Methods

In [ ]:
def get_cnn():
  cnn = tf.keras.models.Sequential()
  return cnn

def get_layer(first=False):
  if (first):
    conv_layer = tf.keras.layers.Conv2D(
      filters=32,
      kernel_size = 3,
      activation = 'relu',
      input_shape = input_shape_default,
      padding='same'
      )
    return conv_layer
  conv_layer = tf.keras.layers.Conv2D(
      filters=64,
      kernel_size = 3,
      activation = 'relu',
      padding='same'
      )
  return conv_layer

def get_pooling():
  max_pooling_layer = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2, padding='valid')
  return max_pooling_layer

def flatten(cnn):
  cnn.add(tf.keras.layers.Flatten())

def add_full_connection(cnn):
  outputLayer = tf.keras.layers.Dense(units=128, activation='relu')
  cnn.add(outputLayer)

def add_output_layer(cnn):
  outputLayer = tf.keras.layers.Dense(units=1, activation='sigmoid')
  cnn.add(outputLayer)

def build_full_model(num_layers=2, shoudl_compile=True):
  cnn = get_cnn()
  for i in range(num_layers):
    first = i == 0
    layer = get_layer(first)
    pooling = get_pooling()
    cnn.add(layer)
    cnn.add(pooling)
  flatten(cnn)
  add_full_connection(cnn)
  add_output_layer(cnn)
  if shoudl_compile:
    cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return cnn

def train(cnn, train_set,test_set, epochs):
  cnn.fit(x= train_set, validation_data= test_set, epochs=epochs)

## Build, Training and Evaluating

In [ ]:
cnn = build_full_model(num_layers=3)
train(cnn, train_set,test_set, epochs_default)

## Predictions

In [ ]:
def predict_item(model, image_path, target_size):
  image_predict = image.load_img(image_path, target_size= target_size)
  image_predict = image.img_to_array(image_predict)
  image_predict = np.expand_dims(image_predict,axis=0)  # Convert single image to a batch.
  predictions = model.predict(image_predict)
  return predictions

def predict_class(train_set, predictions):
  indices = train_set.class_indices
  reverse_indices = {}
  for x, y in indices.items():
    reverse_indices[str(y)] = x
    idx = int(predictions[0][0])
  return reverse_indices[str(idx)]

def predict_many(model, train_set, folder_path, target_size, expected_class):
  images_paths = glob.glob(folder_path)
  predictions_list = []
  for image_path in images_paths:
    predictions = predict_item(model, image_path, target_size)
    class_predicted = predict_class(train_set, predictions)
    prediction = {
        'expected':expected_class.lower(),
        'found': class_predicted.lower(),
        'got_it_rigth': expected_class.lower() == class_predicted.lower()
        }
    predictions_list.append(prediction)
  return predictions_list

def print_predictions(predictions_list):
  rights = filter(lambda item: item['got_it_rigth'] == True, predictions_list)
  _percent_right = len(list(rights)) / len(predictions_list)
  print("% righs {}".format(_percent_right * 100))
  for idx, item in enumerate(predictions_list):
    print("#{} Expected: {} - Found: {} - Right? {}".format(idx,item['expected'], item['found'], item['got_it_rigth']))

In [ ]:
validation_cows = validaton_folder + '/cows/*'
validation_horses = validaton_folder + '/horses/*'

In [ ]:
cows_result = predict_many(cnn, train_set, validation_cows, target_size_default,'cows')
horses_result = predict_many(cnn, train_set, validation_horses, target_size_default,'horses')

In [ ]:
print_predictions(cows_result)

In [ ]:
print_predictions(horses_result)